In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
class Lemmatizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, text):
        return [self.wnl.lemmatize(t) for t in word_tokenize(text)]

data_train = pd.read_csv("texts_train_nn.csv")
texts_train = data_train["text"]
vec = CountVectorizer(strip_accents = 'unicode')
vec.fit(texts_train.values)

CountVectorizer(strip_accents='unicode')

In [ ]:
class AiWritingDataset(Dataset):
    def __init__(self, csv_file, vec):
        self.data = pd.read_csv(csv_file)
        self.vectorizer = vec

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        if isinstance(idx, int):
          sub = self.data.iloc[[idx]]
        else:
          sub = self.data.iloc[idx]
        y = sub["y"].to_numpy()
        X = sub["text"]

        return torch.tensor(self.vectorizer.transform(X).toarray(), dtype=torch.float32), torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [ ]:
train_data = AiWritingDataset("texts_train_nn.csv", vec)
test_data = AiWritingDataset("texts_test_nn.csv", vec)
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_data, batch_size=256, shuffle=False, num_workers=0)

In [ ]:
from collections import OrderedDict

class DenseNetwork(torch.nn.Module):
    def __init__(self, in_features, hidden_size, n_classes, n_layers, dropout, activation=nn.ReLU):
        super().__init__()

        self.layers = n_layers
        self.layer_size = hidden_size
        self.in_features = in_features
        self.activation = activation
        l = []
        if n_layers == 1:
          l.append(('linear0', nn.Linear(self.in_features, n_classes)))
        else:
          l.append(('linear0', nn.Linear(self.in_features, self.layer_size)))
          l.append(('relu0', self.activation()))
          for i in range(1, self.layers - 1):
            l.append((f'linear{i}', nn.Linear(self.layer_size, self.layer_size)))
            l.append((f'relu{i}', self.activation()))
            l.append((f'dropout{i}', nn.Dropout(p=dropout)))
          l.append(('linear_final', nn.Linear(self.layer_size, n_classes)))
          l.append(('sigmoid', nn.Sigmoid()))
        self.layers = nn.Sequential(OrderedDict(l))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.layers(x)

In [ ]:
import tqdm.notebook as tqdm
from functools import partial

def training_loop(n_epochs, test_every_it, network, loss_fn, optimizer, dl_train, dl_test, device):
    train_losses, test_losses, train_accuracies, test_accuracies = [], [], [], []
    for epoch in (pbar := tqdm.tqdm(range(n_epochs), total=n_epochs, leave=False)):

        network.train()

        for data, labels in tqdm.tqdm(dl_train, total=len(dl_train), leave=False):
            optimizer.zero_grad()
            loss = loss_fn(network(data.to(device)), labels.to(device))
            loss.backward()
            optimizer.step()

        if epoch % test_every_it == 0 or epoch == n_epochs - 1:
            network.eval()

            with torch.no_grad():
                losses_train = 0
                accuracies_train = 0

                for data, labels in dl_train:
                    train_out = network(data.to(device))
                    pred = train_out > 0.5
                    losses_train += loss_fn(train_out, labels.to(device)).item()
                    accuracies_train += torch.sum(pred == labels.to(device)).item() / labels.shape[0]

                losses_train /= len(dl_train)
                accuracies_train /= len(dl_train)
                accuracies_train *= 100

                losses_test = 0
                accuracies_test = 0

                for data, labels in dl_test:
                    test_out = network(data.to(device))
                    pred = test_out > 0.5
                    losses_test += loss_fn(test_out, labels.to(device)).item()
                    accuracies_test += torch.sum(pred == labels.to(device)).item() / labels.shape[0]
                losses_test /= len(dl_test)
                accuracies_test /= len(dl_test)
                accuracies_test *= 100

                test_losses.append(losses_test)
                test_accuracies.append(accuracies_test)
                train_losses.append(losses_train)
                train_accuracies.append(accuracies_train)

            pbar.set_description(
                'Loss (Train/Test): {0:.3f}/{1:.3f}. Accuracy, % (Train/Test): {2:.2f}/{3:.2f}\n'.format(
                    train_losses[-1], test_losses[-1], train_accuracies[-1], test_accuracies[-1]
                )
            )

    return train_losses, test_losses, train_accuracies, test_accuracies

In [ ]:
net = DenseNetwork(len(train_data.vectorizer.vocabulary_), 64, 1, 4, 0.1)
print(net)

device = torch.device('cpu')
if torch.cuda.is_available():
    device = torch.device('cuda', 0)

net = net.to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters())

DenseNetwork(
  (layers): Sequential(
    (linear0): Linear(in_features=128768, out_features=64, bias=True)
    (relu0): ReLU()
    (linear1): Linear(in_features=64, out_features=64, bias=True)
    (relu1): ReLU()
    (dropout1): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=64, out_features=64, bias=True)
    (relu2): ReLU()
    (dropout2): Dropout(p=0.1, inplace=False)
    (linear_final): Linear(in_features=64, out_features=1, bias=True)
    (sigmoid): Sigmoid()
  )
)


In [ ]:
optimizer = torch.optim.Adam(net.parameters())

In [ ]:
train_losses, test_losses, train_accs, test_accs = training_loop(
    n_epochs=10, test_every_it=1, network=net, loss_fn=loss_fn,
    optimizer=optimizer, dl_train=train_dataloader, dl_test=test_dataloader, device=device
)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

...

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.plot(train_accs, label="Тренировочная выборка")
ax.plot(test_accs, label="Тестовая выборка")
ax.set_xlabel("Эпоха")
ax.set_ylabel("Точность")
ax.grid(True)
ax.legend()

fig.text(
    0.0, 0.0, str(datetime.now()),
    fontsize=10, color='gray', alpha=0.6,
)

fig.tight_layout()
plt.savefig('bow_bb.png', bbox_inches='tight', dpi=100)
plt.show()

In [ ]:
torch.save(net.state_dict(), "bow_sigmoid.pth")